### Import thư viện

In [1]:
import os
import json
import random

In [2]:
import pandas as pd
import translators as ts

<em>Vui lòng không chạy code này !</em>

### Giữ lại một số lượng file phù hợp

Nhóm sử dụng file train.csv có dung lượng > 1.2GB chứa các tin tức từ 2 kênh thông tin online CNN và Daily Mail. Do hạn chế về việc lưu trữ trên github cũng như các đơn vị tính toán (RAM) trên laptop cá nhân, nhóm sử dụng 100.000 dòng đầu tiên của file csv (tương ứng 100000 bài báo tiếng anh) để làm dữ liệu thực hiện project này.

In [3]:
data = pd.read_csv("../../train.csv", nrows=100000)

In [4]:
data

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...
...,...,...,...
99995,0cd7ec4013b0a6dbbc9f15ce8a7ff757db5b8ad9,"By . Deni Kirkova . PUBLISHED: . 05:54 EST, 1 ...",Eleven innocent princesses raunch up as Miley ...
99996,0cd84db8ebc91821d07d910e830d8710021a2d6a,A furious Harvard Business School professor ha...,Ben Edelman is an associate professor of busin...
99997,0cd875596eada9164e7d349d654697c342020b65,"By . Daily Mail Reporter . Lewis Dale, 17, is ...","Lewis Dale, 17, weeps in the dock as he is tol..."
99998,0cd9048c654458e89b9f2a219da1b4d12bc6550f,By . Daily Mail Reporter . PUBLISHED: . 11:09 ...,CCTV shows driver chase a man on a scooter and...


Thực hiện tách từng file .txt nhỏ, phục vụ cho việc lưu trữ, đẩy file lên github với id đại diện cho tên file

In [5]:
# Tạo thư mục để lưu file TXT nếu chưa tồn tại
output_folder = '../data_summarize'  # Thay đổi tên thư mục nếu cần
os.makedirs(output_folder, exist_ok=True)

In [6]:
# Lưu từng dòng dữ liệu thành file TXT
for index, row in data.iterrows():
    record = {
        'article': row['article'].replace('\\"', "'").replace('\n', ''),
        'highlights': row['highlights'].replace('\\"', "'").replace('\n', '')
    }

    output_file_path = os.path.join(output_folder, f"{row['id']}.txt")
    
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(record, output_file, ensure_ascii=False, indent=2)

print("Done!")

Done!


### Thực hiện translate phần summary tạo target cho mô hình translate

Với phần translate, nhóm sử dụng module translators cũng như đã thử nghiệm một số mô hình pretrained, kết quả dịch không đúng nghĩa, sát nghĩa,... dịch thiếu tokens,... nên nhóm thực hiện gán nhãn bằng tay với công cụ hỗ trợ: google dịch, gpt, poe.com, ...

Do nguồn nhân lực và thời gian còn hạn chế, nhóm chỉ gán nhãn 1000 file bất kỳ để sử dụng cho quá tình fine-tuning model translate.

In [8]:
# Tạo thư mục để lưu file TXT nếu chưa tồn tại
summarize_folder = '../data_summarize'
translate_folder = '../data_translate'  # Thay đổi tên thư mục nếu cần
os.makedirs(translate_folder, exist_ok=True)

In [14]:
files = os.listdir(summarize_folder)

In [16]:
random_files = random.sample(files, 1000)
for file_name in random_files:
    with open(f"{summarize_folder}/{file_name}", 'r', encoding='utf-8') as file:
        content = json.load(file)
        translate_summary = ts.translate_text(content["highlights"], # sử dụng google để dịch
                                            from_language='en', to_language='vi', 
                                            translator='google')
        content["vi"] = translate_summary
        with open(f"{translate_folder}/{file_name}", 'w', encoding='utf-8') as output_file:
            json.dump(content, output_file, ensure_ascii=False, indent=2)
print("Done!")

Done!


In [7]:
import os
import json
import random
from docx import Document

In [8]:
# Tạo một tệp Word mới
doc = Document()

In [9]:
files = os.listdir("../data_translate/")
len(files)

1024

---

In [57]:
for file_name in files[450:550]:
    print(file_name)
    # Đọc nội dung từ trường "highlights" trong tệp JSON
    with open(f"../data_translate/{file_name}", 'r', encoding='utf-8') as file:
        content = json.load(file)
        highlights = content["highlights"]

    # Thêm nội dung vào tệp Word
    doc.add_heading(file_name, level=1)
    doc.add_paragraph(highlights)

# Lưu tệp Word
doc.save("../../highlights.docx")

5e11c722013f7b569444dd296fc79531b43bc3ae.txt
5e8c882030ac996c0b01652a9aa32ed9ab193e40.txt
5f00573a36baf359c3a0c4699a6832a0fa9a7146.txt
5fa85f15f9a59fe51250dc800b4151690ab3dd8b.txt
5fd1ef0a9996ac92554baff2e33ef2764082854e.txt
6053b60608f83887d5e48ed814a68f8d7988568f.txt
607a9d028edfca0480de71ba81b2aae0db900c8d.txt
608b86ddf4f5fce767e880e476fc4c221c191117.txt
60a9f40f38ae06614f7cb91e76fcda4935550ef7.txt
60d10da72917d89b5f4962d74e056d65782d6f1f.txt
611373287e74d77c448d1b1a3ad9f7fc766a57b9.txt
612cd045f630397a0996c55fdecbafa6f9be4d34.txt
61395afbbddec1b3cd9e244a85b1bb9c3775cb5d.txt
613d7b9e813ef8cadbaf9da6a2742d2552422095.txt
619801da91de14de2e13c04ce074ea4161d33853.txt
61ea688df976460106056499ad880745b0304795.txt
62099b902e0f2733b501c7012ee53e5a470d1d51.txt
622b501bcd35dfe68b9a5909c9de3f5fc8814c2d.txt
6247df79ed1be956b1ca91e11e8f52fa4654e58f.txt
62495d2034a2e3dd67c8e82b1b4d2a70cbee7c56.txt
624e0c0d7f57aabd15258913cb36623fce2f3610.txt
625fec1164d0b91420ba92905fe5f75d5baabac6.txt
633d1241d1

JSONDecodeError: Expecting ',' delimiter: line 4 column 307 (char 5840)

In [49]:
translate_folder = "../data_translate/"
docx_folder = "../../sub.docx"

In [50]:
# Đọc nội dung từ tệp highlights.docx
with open(docx_folder, 'rb') as docx_file:
    doc = Document(docx_file)
    for paragraph in doc.paragraphs:
        # Tìm đoạn văn bản có tiêu đề (nếu có) và nội dung
        if paragraph.style.name.startswith('Heading'):
            file_name = paragraph.text
        else:
            txt_path = os.path.join(translate_folder, file_name)
            # Đọc nội dung từ file JSON đã có
            with open(txt_path, 'r', encoding='utf-8') as existing_file:
                existing_content = json.load(existing_file)

            # Ghi lại nội dung của phần "vi" từ tệp highlights.docx vào tệp .txt
            existing_content["vi"] = paragraph.text

            # Ghi nội dung đã cập nhật vào tệp .txt
            with open(txt_path, 'w', encoding='utf-8') as updated_file:
                json.dump(existing_content, updated_file, ensure_ascii=False, indent=2)

            print(f"Đã cập nhật nội dung từ highlights.docx vào {file_name} thành công.")


Đã cập nhật nội dung từ highlights.docx vào 00061f5113a7e6334310f88a12c54d87af483347.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 0016bfe904685ce40f094b07c294c2065dd3a194.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 0030ea36432cd0a5b36484bb603b3755d2fd0e70.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 0037acc8ecedb7b9ad3dbc87e229cec3eeea2bd6.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 00413c507d28a0a8fae46f6ceb0dd43fd8da91be.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 0055114867593ed8c267876b2d4bf0d2f5a97b88.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 00657a4cd247ea033769f76e67a6877c85272366.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 0075795e722ebab52dca94aa77e57d5697f3b5bf.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 00a88dc9b0180f451a5da788b7217f6f79ad7a72.txt thành công.
Đã cập nhật nội dung từ highlights.docx vào 00b4620df3af0867c75d25f09ded465961d0606d.txt th

---